<a href="https://kaggle.com/kernels/welcome?src=https://github.com/mayait/ClaseAnalisisDatos/blob/main/nlp/2022_Twitter_Data_Scrapper.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" width=120 /></a>

<a href="https://colab.research.google.com/github/mayait/ClaseAnalisisDatos/blob/main/nlp/2022_Twitter_Data_Scrapper.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width=120 /></a>

In [4]:
#@title Nombre del estudiante
Estudiante = "" #@param {type:"string"}
Código = "" #@param {type:"string"}



# "Datascraping con Twitter"


Uno de los superpoderes de la AI/ML es que nos permite entender el mundo desde la perspectiva de los datos.

Twitter es una plataforma que proporciona datos extensos en lenguaje natural, en base a los cuales se puede explorar mucho.

In [5]:
!pip install tweepy textblob wordcloud

In [6]:
# importing libraries
import pandas as pd
import tweepy
import time
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import clear_output
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS

Para acceder a la API de Twitter, necesitaremos generar las claves. 
Estas claves se encuentran en la configuración de la aplicación de Twitter en la pestaña Claves y tokens de acceso:

[ https://apps.twitter.com/app/new]( https://apps.twitter.com/app/new)

In [7]:
#@title Credenciales de Twitter
TWITTER_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPY7WQEAAAAArv14iz4pbeglBLonfhMIReDMbaI%3DE4RZce3DXjAoAI39NWFGA5MQUguGN3FPVE1KcfrLIwtbuRGpft"
client = tweepy.Client(TWITTER_BEARER_TOKEN)

### Tweepy nos permite conectarnos directamente al API de Twitter y obtener tweets
En este ejemplo nos decargamos 10 Tweets de Ecuador

In [25]:

# La función search_recent_tweets() devuelve un objeto de tipo Response, que contiene los tweets en su atributo data.
# El parámetro max_results indica el número máximo de tweets que se devolverán en la respuesta.

response = client.search_recent_tweets("Ecuador -is:retweet", max_results=10)
tweets = response.data

# Iterar a través de los tweets e imprimir el texto
for tweet in response.data:
    print(tweet.text)



En democracia ningún pais de America Latina ha logrado disminuir la delincuencia, la violencia, el narcotrafico. Mira el informe que salió en @faznet sobre el decomiso de cocaína en Berlin escondido en plátanos que venían de Ecuador. https://t.co/wwgXAx7MjK
@AlberdianoArg @laderechadiario Y pero el caso de Ecuador?
@EcuavisaInforma Bien #Chile, y para cuando #Ecuador. 🇪🇨
@PoliciaEcuador 
@AsambleaEcuador
@Jajajaland Ecuador esta de luto
En #Ecuador 🇪🇨 la locución "el rato de los ratos" significa 'la hora de la verdad, el momento decisivo'.

#DiccionarioDeAmericanismos 📖
@eluniversocom Asamblea constituyente ha beneficiado al Ecuador? La respuesta está en las calles! Y noticias que todos los días Uds ponen en sus portadas
#Ecuador #Quito #Guayaquil #Manta #Esmeraldas #Cuenca #Loja #PatriotasEcuatorianos defendamos las #Democracia contra la #AsambleaEcuador, los Iza.  Tambien para exigir cambios a @LassoGuillermo mano fuerte contra la #Corrupcion #criminales.
#Autoconvocatoria #Revocator

### Tambien podemos buscar datos de un usuario especifico
Necesitamos el ID del usuario, puedes buscarlo aquí

https://tweeterid.com/

@usfq_ecuador => 66436490

In [32]:
id = '66436490'
tweets = client.get_users_tweets(id=id, tweet_fields=['context_annotations','created_at','geo'], max_results=10)
for tweet in tweets.data:
    print(tweet.created_at, tweet.text)

2023-04-10 22:19:35+00:00 ¿Listos para vivir múltiples experiencias?
Atrévanse a descubrir el siguiente nivel de innovación tecnológica.
NeoCom 2023: Comunicación, IA y Metaverso.

https://t.co/P7UdKbcLTS

#neocom #neocom2023 #inteligenciaartificial #ai #metaverso #comunicacion #usfq #seminario  #atrevia https://t.co/ujl4gN5yMI
2023-04-10 22:13:57+00:00 Inicia la conferencia de @LeoLopezLujan en la #USFQ; “En busca de la antigua #Tenochtitlan” y la arqueología en la Ciudad de #México. Una magnífica visita y nuevos conocimientos para fortalecer la estrecha relación entre 🇪🇨 y 🇲🇽 en la academia. Bienvenido @LeoLopezLujan 👏🏻 https://t.co/rzuAwheyaq
2023-04-10 18:10:54+00:00 RT @USFQ_Derecho: ✅ I ENCUENTRO NACIONAL DE POLITICA CRIMINAL, DERECHO PENAL Y EJECUCIÓN DE PENAS.

📅 18, 19, 20 abril 2023
📍 Teatro Calder…
2023-04-10 17:43:04+00:00 El miércoles 29 de marzo invitamos a los graduados de Relaciones Internacionales de los últimos 5 años de la USFQ. El encuentro permitió identificar opor

## Una función que extrae Tuits de un tema y los pasa a un DataFrame

In [28]:
# Función para obtener tweets recientes

def getTweets(keyword, num_tweets):
    # Consulta de búsqueda que incluye el término de búsqueda, excluye retweets y selecciona tweets en español
    query = f'{keyword} -is:retweet lang:es'

    # Uso del paginador de Tweepy para obtener más de 100 tweets recientes a través de la API de Twitter
    tweets = []
    for tweet in tweepy.Paginator(client.search_recent_tweets,
                                  query=query,
                                  tweet_fields=['id', 'created_at', 'public_metrics', 'text', 'source'],
                                  max_results=100).flatten(limit=num_tweets):

        tweets.append(tweet)

    result = []

    # Función para desempaquetar la lista de tweets en un DataFrame
    for tweet in tweets:
        result.append({'id': tweet.id,
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'source': tweet.source,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                       })

    # Crear un DataFrame y convertir la columna 'created_at' a objetos datetime
    df = pd.DataFrame(result)
    df['created_at'] = pd.to_datetime(df['created_at'])
    
    # Devolver la lista de tweets
    return tweets


In [35]:
tweets_claro = getTweets("ClaroEcua", 100000)

In [22]:
tweets_claro.to_excel("tweets_ecuador.xlsx")

ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.

# Ejercicio
Realiza una busqueda en twitter de un termino en español y otro en inglés, exportalos a Excel en archivos separados.

nlp/2022_Twitter_Data_Scrapper.ipynb

> Bloque con sangría

